In [1]:
import numpy as np

import os
#Choose GPU 0 as a default
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

import sys
sys.path.append('/home/zsteineh/cnn_hilbert/cnn_hilbert_workspace')
import hilbert_DL_utils
from hilbert_DL_utils import load_data

In [2]:
fold = 1
model_dir = '/data1/users/gsquist/state_decoder/accuracy_outputs/a0f66459/class_ssl/'
model_name = 'sig_tran_model_htnet_fold_'+str(fold)+'.h5'

model_fname = model_dir + model_name

norm_rate = 0.25
wrist_lp = '/data1/users/stepeter/cnn_hilbert/ecog_data/xarray/'
pats_ids_in = ['a0f66459']
test_day = 'last'
n_chans_all=64
tlim=[-1,1]
n_folds = 1

optimizer='adam'
loss='binary_crossentropy'
patience = 15
early_stop_monitor='val_loss'
epochs=64
sp = '/home/zsteineh/ez_ssl_results/'
chckpt_path = sp+'checkpoint_gen_tl_'+pats_ids_in[0]+'_fold'+str(fold)+'.h5'

In [3]:
X,y,x_test,y_test,sbj_order_all,sbj_order_test_last = load_data(pats_ids_in, wrist_lp,
                                                              n_chans_all=n_chans_all,
                                                              test_day=test_day, tlim=tlim)

100%|██████████| 1/1 [00:00<00:00,  2.39it/s]

Data loaded!


In [4]:
nb_classes = len(np.unique(y))
order_inds = np.arange(len(y))
np.random.shuffle(order_inds)
X = X[order_inds,...]
y = y[order_inds]
order_inds_test = np.arange(len(y_test))
np.random.shuffle(order_inds_test)
# X_test = X_test[order_inds_test,...]
# y_test = y_test[order_inds_test]
y2 = np_utils.to_categorical(y-1)
y_test2 = np_utils.to_categorical(y_test-1)
X2 = np.expand_dims(X,1)
X_test2 = np.expand_dims(x_test,1)

split_len = int(X2.shape[0]*0.2)
last_epochs = np.zeros([n_folds,2])

val_inds = np.arange(0,split_len)+(0*split_len)
train_inds = np.setdiff1d(np.arange(X2.shape[0]),val_inds) #take all events not in val set

x_train = X2[train_inds,...]
y_train = y2[train_inds]
x_val = X2[val_inds,...]
y_val = y2[val_inds]

In [5]:
pretask_model = tf.keras.models.load_model(model_fname)
pretask_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 64, 501)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 64, 501)        512       
_________________________________________________________________
lambda (Lambda)              (None, 8, 64, 501)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 64, 501)        32        
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 16, 1, 501)        1024      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 1, 501)        64        
_________________________________________________________________
activation (Activation)      (None, 16, 1, 501)        0     

In [6]:
x = pretask_model.layers[-3].output
# x = Flatten(name = 'flatten2')(x)
x = Dense(nb_classes, name = 'dense', kernel_constraint = max_norm(norm_rate))(x)
softmax = Activation('softmax', name = 'softmax')(x)

transfer_model = Model(inputs=pretask_model.input, outputs=softmax)

# Set only last 3 layers to be trainable
for l in transfer_model.layers:
    l.trainable = False
for l in transfer_model.layers[-3:]:
    l.trainable = True #train last 3 layers
    
transfer_model.get_layer('depthwise_conv2d').trainable = True

transfer_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 64, 501)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 64, 501)        512       
_________________________________________________________________
lambda (Lambda)              (None, 8, 64, 501)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 64, 501)        32        
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 16, 1, 501)        1024      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 1, 501)        64        
_________________________________________________________________
activation (Activation)      (None, 16, 1, 501)        0     

In [7]:
transfer_model.compile(loss=loss, optimizer=optimizer, metrics = ['accuracy'])
checkpointer = ModelCheckpoint(filepath=chckpt_path,verbose=1,save_best_only=True)
early_stop = EarlyStopping(monitor=early_stop_monitor, mode='min',
                               patience=patience, verbose=0)

In [8]:
h = transfer_model.fit(x_train, y_train, batch_size = 16, epochs = epochs, 
                        verbose = 2, validation_data=(x_val, y_val),
                        callbacks=[checkpointer,early_stop])

Epoch 1/64

Epoch 00001: val_loss improved from inf to 0.68901, saving model to /home/zsteineh/ez_ssl_results/checkpoint_gen_tl_a0f66459_fold1.h5
48/48 - 1s - loss: 0.6972 - accuracy: 0.5258 - val_loss: 0.6890 - val_accuracy: 0.5132
Epoch 2/64

Epoch 00002: val_loss did not improve from 0.68901
48/48 - 0s - loss: 0.6436 - accuracy: 0.6235 - val_loss: 0.7966 - val_accuracy: 0.5291
Epoch 3/64

Epoch 00003: val_loss improved from 0.68901 to 0.62557, saving model to /home/zsteineh/ez_ssl_results/checkpoint_gen_tl_a0f66459_fold1.h5
48/48 - 0s - loss: 0.5502 - accuracy: 0.7160 - val_loss: 0.6256 - val_accuracy: 0.5714
Epoch 4/64

Epoch 00004: val_loss improved from 0.62557 to 0.46912, saving model to /home/zsteineh/ez_ssl_results/checkpoint_gen_tl_a0f66459_fold1.h5
48/48 - 0s - loss: 0.4768 - accuracy: 0.7781 - val_loss: 0.4691 - val_accuracy: 0.7619
Epoch 5/64

Epoch 00005: val_loss improved from 0.46912 to 0.43751, saving model to /home/zsteineh/ez_ssl_results/checkpoint_gen_tl_a0f66459_fo

Epoch 46/64

Epoch 00046: val_loss did not improve from 0.30640
48/48 - 0s - loss: 0.3164 - accuracy: 0.8798 - val_loss: 0.3120 - val_accuracy: 0.8995
Epoch 47/64

Epoch 00047: val_loss did not improve from 0.30640
48/48 - 0s - loss: 0.2923 - accuracy: 0.8956 - val_loss: 0.3163 - val_accuracy: 0.8730
Epoch 48/64

Epoch 00048: val_loss improved from 0.30640 to 0.30340, saving model to /home/zsteineh/ez_ssl_results/checkpoint_gen_tl_a0f66459_fold1.h5
48/48 - 0s - loss: 0.2945 - accuracy: 0.8930 - val_loss: 0.3034 - val_accuracy: 0.8995
Epoch 49/64

Epoch 00049: val_loss did not improve from 0.30340
48/48 - 0s - loss: 0.2963 - accuracy: 0.8838 - val_loss: 0.3151 - val_accuracy: 0.9048
Epoch 50/64

Epoch 00050: val_loss improved from 0.30340 to 0.30315, saving model to /home/zsteineh/ez_ssl_results/checkpoint_gen_tl_a0f66459_fold1.h5
48/48 - 0s - loss: 0.2951 - accuracy: 0.8904 - val_loss: 0.3031 - val_accuracy: 0.8942
Epoch 51/64

Epoch 00051: val_loss did not improve from 0.30315
48/48 -

In [9]:
transfer_model.load_weights(chckpt_path)
acc_lst = []
preds = transfer_model.predict(x_train).argmax(axis = -1) 
acc_lst.append(np.mean(preds == y_train.argmax(axis=-1)))
preds = transfer_model.predict(x_val).argmax(axis=-1)
acc_lst.append(np.mean(preds == y_val.argmax(axis=-1)))
preds = transfer_model.predict(X_test2).argmax(axis = -1)
acc_lst.append(np.mean(preds == y_test2.argmax(axis=-1)))

print(np.asarray(acc_lst))

[0.91017173 0.8994709  0.84139785]
